In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [59]:
import numpy as np
import tensorflow as tf
import datetime
import torch
import torchvision
import torchvision.datasets as datasets

from matplotlib import pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.datasets import mnist


In [85]:
batch_size_train = 64
batch_size_test = 1000

In [60]:
#Download the dataset
train_loader = torch.utils.data.DataLoader(datasets.MNIST('/files/', train=True, download=True,
                            transform=torchvision.transforms.Compose([
                            torchvision.transforms.ToTensor(),
                            torchvision.transforms.Normalize(
                            (0.1307,), (0.3081,))])), 
                            batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(datasets.MNIST('/files/', train=False, download=True,
                            transform=torchvision.transforms.Compose([
                            torchvision.transforms.ToTensor(),
                            torchvision.transforms.Normalize(
                            (0.1307,), (0.3081,))])),
                            batch_size=batch_size_test, shuffle=True)

9913344it [00:00, 95017100.95it/s]                             


Extracting /files/MNIST\raw\train-images-idx3-ubyte.gz to /files/MNIST\raw



29696it [00:00, 11913347.83it/s]         

Extracting /files/MNIST\raw\train-labels-idx1-ubyte.gz to /files/MNIST\raw



1649664it [00:00, 84817930.47it/s]         


Extracting /files/MNIST\raw\t10k-images-idx3-ubyte.gz to /files/MNIST\raw



5120it [00:00, 10284883.37it/s]         

Extracting /files/MNIST\raw\t10k-labels-idx1-ubyte.gz to /files/MNIST\raw



In [79]:
n_epochs = 3
learning_rate = 0.0001
momentum = 0.5
log_interval = 10

In [65]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [74]:
# define cnn model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [75]:
# define model
model = Net()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [76]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [77]:
def train(net, epoch, train_loader):
  net.train() # set model into training mode.

  for batch_idx, (data, labels) in enumerate(train_loader):
    optimizer.zero_grad()
    output = net(data)
    loss = F.nll_loss(output, labels)
    loss.backward()
    optimizer.step()

    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [78]:
def test(net, test_loader):
  net.eval() # Set model into evaluation mode, no training.
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, labels in test_loader:
      output = net(data)
      test_loss += F.nll_loss(output, labels, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(labels.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [80]:
test(model, test_loader)
for epoch in range(1, n_epochs + 1):
  train(model, epoch, train_loader)
  test(model, test_loader)

C:\Users\olive\AppData\Local\Temp/ipykernel_15072/3135501261.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)



Test set: Avg. loss: 2.3089, Accuracy: 1150/10000 (12%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.326469
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.328657
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.297970
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.303507
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.279013
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.298615
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.280208
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.303261
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.268538
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.291901
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.248954
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.250186
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.239230
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.237544
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.194301
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.257569
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.152229
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.131834
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.090412
Train Epoch: 1 [12160

In [84]:
torch.save(model, './CNN_model')

In [88]:
from torchvision import transforms

transform = transforms.Compose([
                    transforms.CenterCrop(28),
                    transforms.Grayscale(num_output_channels=1),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5), (0.5))])

svhn_train_loader = torch.utils.data.DataLoader(torchvision.datasets.SVHN(root='/files/', split='train', download=True,  transform=transform),
                                                batch_size=batch_size_test, shuffle=True)
svhn_test_loader = torch.utils.data.DataLoader(torchvision.datasets.SVHN(root='/files/', split='test', download=True,  transform=transform),
                                               batch_size=batch_size_test, shuffle=True)

182041600it [00:23, 7674766.47it/s]                                


64275456it [00:12, 5191665.10it/s]                               


In [89]:
mnistCNN = torch.load('./CNN_model')

In [90]:
test(model, svhn_test_loader)
model.train()
n_epochs = 50 # Will abort training if/when it peaks
for epoch in range(1, n_epochs + 1):
  train(model, epoch, svhn_train_loader)
  test(model, svhn_test_loader)

C:\Users\olive\AppData\Local\Temp/ipykernel_15072/3135501261.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
C:\Users\olive\anaconda3\envs\D7047E_ADL\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.2991, Accuracy: 5087/26032 (20%)

Train Epoch: 1 [0/73257 (0%)]	Loss: 2.469324
Train Epoch: 1 [10000/73257 (14%)]	Loss: 2.451465
Train Epoch: 1 [20000/73257 (27%)]	Loss: 2.323581
Train Epoch: 1 [30000/73257 (41%)]	Loss: 2.309155
Train Epoch: 1 [40000/73257 (54%)]	Loss: 2.250917
Train Epoch: 1 [50000/73257 (68%)]	Loss: 2.244905
Train Epoch: 1 [60000/73257 (81%)]	Loss: 2.205399
Train Epoch: 1 [70000/73257 (95%)]	Loss: 2.218188

Test set: Avg. loss: 2.1555, Accuracy: 6395/26032 (25%)

Train Epoch: 2 [0/73257 (0%)]	Loss: 2.223496
Train Epoch: 2 [10000/73257 (14%)]	Loss: 2.231782
Train Epoch: 2 [20000/73257 (27%)]	Loss: 2.213721
Train Epoch: 2 [30000/73257 (41%)]	Loss: 2.181494
Train Epoch: 2 [40000/73257 (54%)]	Loss: 2.208709
Train Epoch: 2 [50000/73257 (68%)]	Loss: 2.179131
Train Epoch: 2 [60000/73257 (81%)]	Loss: 2.207169
Train Epoch: 2 [70000/73257 (95%)]	Loss: 2.192753

Test set: Avg. loss: 2.1220, Accuracy: 7187/26032 (28%)

Train Epoch: 3 [0/73257 (0%)]	Loss: 